# PAST Model: Getting Started

This notebook will help you get started with the `PastModel` model.
You will:
- Download a pretrained model from HuggingFace
- Load and process audio
- Run the model (encode & decode)
- Listen to original and reconstructed audio
- evaluate the model using PESQ and SI-SNR metrics

In [ ]:
# Optional: install dependencies
# !pip install -r requirements.txt

In [ ]:
import sys

sys.path.insert(0, '.')

### 🔧 Load the model

In [ ]:
from past.models.past_model import PastModel

model_name = 'PAST'  # can be also 'PAST_streamable'
model = PastModel.from_pretrained(model_name)

print("Sample rate:", model.sample_rate)

### 🔊 Load an audio file

In [ ]:
import torchaudio


def read_one_wav(path: str, target_sr):
    wav, sr = torchaudio.load(path)
    if sr != target_sr:
        wav = torchaudio.transforms.Resample(sr, target_sr)(wav)
    if wav.shape[0] == 2:
        wav = wav[:1]
    return wav.unsqueeze(0)


# Example path:
audio_path = "assets/1089-134686-0004.flac"
wav = read_one_wav(audio_path, model.sample_rate).to(model.device)

print("Loaded audio shape:", wav.shape)

### 🔄 Run the model (encode + decode)

In [ ]:
import torch

with torch.no_grad():
    codes, scale = model.encode(wav)
    reconstructed = model.decode(codes, scale)

reconstructed = reconstructed[:, :, : wav.shape[2]]

print("Codes shape:", codes.shape)

### 🧩 View token codes

In [ ]:
import pandas as pd

codes_for_print = codes[0, :, :10].cpu().numpy()
T, K = codes_for_print.shape
print(f"Codes shape for printing: {codes_for_print.shape} (T={T}, K={K})")

df = pd.DataFrame(codes_for_print.T, columns=[f"token {i+1}" for i in range(T)])
df.index = [f"time {i+1}" for i in range(K)]

print(df.to_string(float_format="%.2f"))

### 🔁 Listen to original vs reconstructed audio

In [ ]:
from IPython.display import Audio, display

print("Original:")
display(Audio(wav.cpu().numpy().squeeze(), rate=model.sample_rate))

print("Reconstructed:")
display(Audio(reconstructed.cpu().numpy().squeeze(), rate=model.sample_rate))

### 📈 Evaluate Model Performance on a single wav

In [ ]:
from audiocraft.losses.sisnr import SISNR

try:
    from pypesq import pesq
except ImportError:
    print("pypesq is not installed. PESQ metric will not be calculated.")
    pesq = None

sisnr_val = SISNR(sample_rate=model.sample_rate, segment=5, overlap=0.5, epsilon=1e-8)(reconstructed, wav)
pesq_val = pesq(wav.squeeze().cpu().numpy(), reconstructed.squeeze().cpu().numpy(), model.sample_rate) if pesq is not None else None

print(f"## The metrics are calculated for the original and reconstructed audio. ##")
print(f"SI-SNR: {sisnr_val:.2f}")
if pesq_val is not None:
    print(f"PESQ: {pesq_val:.2f}")